In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import tqdm
import sklearn as sk
import os
import random
import math

In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num Physical Devices Available: ", len(physical_devices))
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(tf.__version__)

Num Physical Devices Available:  1
2.8.0


**Constants**


In [9]:
IMG_SIZE = 600
BATCH_SIZE = 64
PATH = './video_data/Real Life Violence Dataset/'

**Make Datatset**


In [4]:
from sklearn.model_selection import train_test_split

In [13]:
def video_to_frames(path):

    cap = cv2.VideoCapture(path)
    rate = math.floor(cap.get(3))

    ImageFrames = []

    while cap.isOpened():
        ID = cap.get(1)

        success, image = cap.read()

        if success:
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            ImageFrames.append(image)
        
        else:
            print("task failed")
            break
    
    return ImageFrames
    

In [12]:
CATEGORIES = ["NonViolence", "Violence"]
X_set = []
y_set = []
for category in os.listdir(PATH):
    path = os.path.join(PATH, category)
    class_num = CATEGORIES.index(category)
    for i, video in enumerate(tqdm(os.listdir(path)[0:350])):
        frames = video_to_frames(path + '/' + video)
        for j, frame in enumerate(frames):
            X_set.append(frame)
            y_set.append(class_num)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './video_data/Real Life Violence Detection'

In [ ]:
def create_dataset():
    dataset = []
    
    for file in tqdm(os.listdir('./video_data/Real Life Violence Dataset/')):
        path = os.path.join('./video_data/Real Life Violence Dataset/', file)
        vid = cv2.resize(cv2.imread(path), (INPUT_SIZE, INPUT_SIZE))



    random.shuffle(dataset)     

In [ ]:
#EffNet = tf.keras.applications.EfficientNetV2B0(include_top=False, weights = 'imageNet', )